# 00 - Schema Quality Check & Data Inventory

This notebook performs comprehensive data quality analysis for the Aadhaar datasets.

**Focus**: 5-15 age group biometric update bottleneck detection.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import glob
import os
from datetime import datetime

pd.set_option('display.max_columns', None)
print("Libraries loaded.")

Libraries loaded.


## 1. Load All Datasets

In [2]:
def load_dataset(pattern):
    """Load and concatenate all CSV files matching a pattern."""
    search_path = os.path.join('../data/raw', pattern)
    files = glob.glob(search_path)
    print(f"Pattern '{pattern}': {len(files)} files")
    if not files:
        return pd.DataFrame()
    dfs = [pd.read_csv(f) for f in files]
    return pd.concat(dfs, ignore_index=True)

print("Loading datasets...")
enrolment = load_dataset('*enrolment*.csv')
demo_update = load_dataset('*demographic*.csv')
bio_update = load_dataset('*biometric*.csv')

print(f"\nEnrolment: {len(enrolment):,} rows")
print(f"Demographic: {len(demo_update):,} rows")
print(f"Biometric: {len(bio_update):,} rows")

Loading datasets...
Pattern '*enrolment*.csv': 3 files
Pattern '*demographic*.csv': 5 files
Pattern '*biometric*.csv': 4 files

Enrolment: 1,006,029 rows
Demographic: 2,071,700 rows
Biometric: 1,861,108 rows


In [3]:
# Parse dates (DD-MM-YYYY format)
def parse_dates(df, col='date'):
    df = df.copy()
    df[col] = pd.to_datetime(df[col], format='%d-%m-%Y', errors='coerce')
    return df

enrolment = parse_dates(enrolment)
demo_update = parse_dates(demo_update)
bio_update = parse_dates(bio_update)

print("Date parsing complete.")
print(f"Enrolment nulls in date: {enrolment['date'].isnull().sum()}")
print(f"Demographic nulls in date: {demo_update['date'].isnull().sum()}")
print(f"Biometric nulls in date: {bio_update['date'].isnull().sum()}")

Date parsing complete.
Enrolment nulls in date: 0
Demographic nulls in date: 0
Biometric nulls in date: 0


## 2. Temporal Coverage Analysis

In [4]:
# Get column names for inspection
print("Enrolment columns:", enrolment.columns.tolist())
print("Demographic columns:", demo_update.columns.tolist())
print("Biometric columns:", bio_update.columns.tolist())

Enrolment columns: ['date', 'state', 'district', 'pincode', 'age_0_5', 'age_5_17', 'age_18_greater']
Demographic columns: ['date', 'state', 'district', 'pincode', 'demo_age_5_17', 'demo_age_17_']
Biometric columns: ['date', 'state', 'district', 'pincode', 'bio_age_5_17', 'bio_age_17_']


In [5]:
# Aggregate daily counts
# Identify numeric columns for aggregation
enrol_num_cols = enrolment.select_dtypes(include=[np.number]).columns.tolist()
demo_num_cols = demo_update.select_dtypes(include=[np.number]).columns.tolist()
bio_num_cols = bio_update.select_dtypes(include=[np.number]).columns.tolist()

enrol_daily = enrolment.groupby('date')[enrol_num_cols].sum().reset_index()
demo_daily = demo_update.groupby('date')[demo_num_cols].sum().reset_index()
bio_daily = bio_update.groupby('date')[bio_num_cols].sum().reset_index()

print(f"Enrolment daily range: {enrol_daily['date'].min()} to {enrol_daily['date'].max()}")
print(f"Demographic daily range: {demo_daily['date'].min()} to {demo_daily['date'].max()}")
print(f"Biometric daily range: {bio_daily['date'].min()} to {bio_daily['date'].max()}")

Enrolment daily range: 2025-03-02 00:00:00 to 2025-12-31 00:00:00
Demographic daily range: 2025-03-01 00:00:00 to 2025-12-29 00:00:00
Biometric daily range: 2025-03-01 00:00:00 to 2025-12-29 00:00:00


In [6]:
# Create total column for each dataset (sum of all numeric columns per row)
enrol_daily['total'] = enrol_daily[enrol_num_cols].sum(axis=1)
demo_daily['total'] = demo_daily[demo_num_cols].sum(axis=1)
bio_daily['total'] = bio_daily[bio_num_cols].sum(axis=1)

# Plot temporal coverage
fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                    subplot_titles=('Enrolment Daily Total', 'Demographic Updates Daily Total', 'Biometric Updates Daily Total'),
                    vertical_spacing=0.08)

fig.add_trace(go.Scatter(x=enrol_daily['date'], y=enrol_daily['total'], mode='lines', name='Enrolment', line=dict(color='#2E86AB')), row=1, col=1)
fig.add_trace(go.Scatter(x=demo_daily['date'], y=demo_daily['total'], mode='lines', name='Demographic', line=dict(color='#A23B72')), row=2, col=1)
fig.add_trace(go.Scatter(x=bio_daily['date'], y=bio_daily['total'], mode='lines', name='Biometric', line=dict(color='#F18F01')), row=3, col=1)

fig.update_layout(height=700, title_text='Temporal Coverage: Daily Counts Over Time', showlegend=False)
fig.update_xaxes(title_text='Date', row=3, col=1)
fig.update_yaxes(title_text='Count')
fig.show()

In [7]:
# Weekly pattern analysis (day of week)
enrol_daily['dow'] = enrol_daily['date'].dt.day_name()
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

dow_avg = enrol_daily.groupby('dow')['total'].mean().reindex(dow_order)

fig = px.bar(x=dow_order, y=dow_avg.values, labels={'x': 'Day of Week', 'y': 'Average Enrolments'},
             title='Weekly Pattern: Average Enrolments by Day of Week',
             color=dow_avg.values, color_continuous_scale='Viridis')
fig.show()

## 3. Geographic Coverage Analysis

In [8]:
# States and districts coverage
print("=== Geographic Coverage ===")
print(f"Enrolment - States: {enrolment['state'].nunique()}, Districts: {enrolment['district'].nunique()}")
print(f"Demographic - States: {demo_update['state'].nunique()}, Districts: {demo_update['district'].nunique()}")
print(f"Biometric - States: {bio_update['state'].nunique()}, Districts: {bio_update['district'].nunique()}")

=== Geographic Coverage ===
Enrolment - States: 55, Districts: 985
Demographic - States: 65, Districts: 983
Biometric - States: 57, Districts: 974


In [9]:
# District mismatch analysis
enrol_districts = set(enrolment['district'].unique())
demo_districts = set(demo_update['district'].unique())
bio_districts = set(bio_update['district'].unique())

common_districts = enrol_districts & demo_districts & bio_districts
all_districts = enrol_districts | demo_districts | bio_districts

print(f"\nCommon districts (in all 3): {len(common_districts)}")
print(f"Total unique districts: {len(all_districts)}")
print(f"Districts missing from at least one dataset: {len(all_districts) - len(common_districts)}")

# Sample problematic districts
only_in_enrol = enrol_districts - (demo_districts | bio_districts)
only_in_demo = demo_districts - (enrol_districts | bio_districts)
only_in_bio = bio_districts - (enrol_districts | demo_districts)

print(f"\nOnly in Enrolment: {len(only_in_enrol)} - Sample: {list(only_in_enrol)[:5]}")
print(f"Only in Demographic: {len(only_in_demo)} - Sample: {list(only_in_demo)[:5]}")
print(f"Only in Biometric: {len(only_in_bio)} - Sample: {list(only_in_bio)[:5]}")


Common districts (in all 3): 948
Total unique districts: 1029
Districts missing from at least one dataset: 81

Only in Enrolment: 33 - Sample: ['Ranga Reddy', 'Ahmednagar', 'Dinajpur Uttar', 'Spsr Nellore', 'Dinajpur Dakshin']
Only in Demographic: 20 - Sample: ['Near Dhyana Ashram', 'Near University Thana', 'Dist : Thane', 'Near meera hospital', 'Bicholim']
Only in Biometric: 11 - Sample: ['BALANGIR', 'Khordha  *', 'Gautam Buddha Nagar *', 'Mahoba *', 'Anugul  *']


In [10]:
# State-level aggregation for heatmap
enrol_by_state = enrolment.groupby('state')[enrol_num_cols].sum().sum(axis=1).reset_index(name='enrol_total')
demo_by_state = demo_update.groupby('state')[demo_num_cols].sum().sum(axis=1).reset_index(name='demo_total')
bio_by_state = bio_update.groupby('state')[bio_num_cols].sum().sum(axis=1).reset_index(name='bio_total')

state_summary = enrol_by_state.merge(demo_by_state, on='state', how='outer').merge(bio_by_state, on='state', how='outer')
state_summary = state_summary.fillna(0).sort_values('enrol_total', ascending=False)

fig = px.bar(state_summary.head(15), x='state', y=['enrol_total', 'demo_total', 'bio_total'],
             title='Top 15 States by Total Activity', barmode='group',
             labels={'value': 'Count', 'variable': 'Dataset'})
fig.update_layout(xaxis_tickangle=-45)
fig.show()

## 4. Age-Group Alignment

In [11]:
# Identify age columns in each dataset
enrol_age_cols = [c for c in enrolment.columns if 'age' in c.lower()]
demo_age_cols = [c for c in demo_update.columns if 'age' in c.lower()]
bio_age_cols = [c for c in bio_update.columns if 'age' in c.lower()]

print("Age columns in Enrolment:", enrol_age_cols)
print("Age columns in Demographic:", demo_age_cols)
print("Age columns in Biometric:", bio_age_cols)

Age columns in Enrolment: ['age_0_5', 'age_5_17', 'age_18_greater']
Age columns in Demographic: ['demo_age_5_17', 'demo_age_17_']
Age columns in Biometric: ['bio_age_5_17', 'bio_age_17_']


In [12]:
# Age group mapping for canonical 5-15 bucket
# Assumption: 5-17 columns are the closest proxy for 5-15
print("\n=== Age Alignment Strategy ===")
print("Canonical bucket: 5-15 (children requiring biometric update)")
print("Mapping:")
print("  - Enrolment 'age_5_17' or similar -> use as proxy for 5-15")
print("  - Demographic 'demo_age_5_17' -> direct mapping")
print("  - Biometric 'bio_age_5_17' -> direct mapping")
print("\nNote: age_0_5 in enrolment has no equivalent in update datasets.")


=== Age Alignment Strategy ===
Canonical bucket: 5-15 (children requiring biometric update)
Mapping:
  - Enrolment 'age_5_17' or similar -> use as proxy for 5-15
  - Demographic 'demo_age_5_17' -> direct mapping
  - Biometric 'bio_age_5_17' -> direct mapping

Note: age_0_5 in enrolment has no equivalent in update datasets.


## 5. Child-Focused Metrics (5-15 Age Group)

In [13]:
# Identify the 5-17 columns dynamically
demo_child_col = [c for c in demo_update.columns if '5' in c and '17' in c]
bio_child_col = [c for c in bio_update.columns if '5' in c and '17' in c]

print(f"Demo child column: {demo_child_col}")
print(f"Bio child column: {bio_child_col}")

if demo_child_col and bio_child_col:
    demo_child_col = demo_child_col[0]
    bio_child_col = bio_child_col[0]
    
    # Aggregate by district
    demo_district = demo_update.groupby(['state', 'district'])[demo_child_col].sum().reset_index()
    bio_district = bio_update.groupby(['state', 'district'])[bio_child_col].sum().reset_index()
    
    child_metrics = demo_district.merge(bio_district, on=['state', 'district'], how='outer').fillna(0)
    child_metrics['bio_demo_ratio'] = child_metrics[bio_child_col] / child_metrics[demo_child_col].replace(0, np.nan)
    child_metrics['bio_demo_gap'] = child_metrics[demo_child_col] - child_metrics[bio_child_col]
    
    print(f"\nChild metrics computed for {len(child_metrics)} districts.")
    print(f"Average bio/demo ratio: {child_metrics['bio_demo_ratio'].mean():.3f}")
    print(f"Districts with ratio < 0.5 (potential bottleneck): {(child_metrics['bio_demo_ratio'] < 0.5).sum()}")
else:
    print("Could not identify child age columns.")

Demo child column: ['demo_age_5_17']
Bio child column: ['bio_age_5_17']

Child metrics computed for 1097 districts.
Average bio/demo ratio: 7.510
Districts with ratio < 0.5 (potential bottleneck): 32


In [14]:
# Top bottleneck districts (lowest bio/demo ratio)
if 'bio_demo_ratio' in child_metrics.columns:
    bottleneck_districts = child_metrics.dropna(subset=['bio_demo_ratio']).nsmallest(20, 'bio_demo_ratio')
    print("\n=== Top 20 Potential Bottleneck Districts (Lowest Bio/Demo Ratio) ===")
    display(bottleneck_districts[['state', 'district', demo_child_col, bio_child_col, 'bio_demo_ratio', 'bio_demo_gap']])


=== Top 20 Potential Bottleneck Districts (Lowest Bio/Demo Ratio) ===


,state,district,demo_age_5_17,bio_age_5_17,bio_demo_ratio,bio_demo_gap
581,Manipur,Kangpokpi,2.0,0.0,0.000000,2.0
707,Orissa,Nabarangapur,1.0,0.0,0.000000,1.0
757,Rajasthan,Balotra,26.0,0.0,0.000000,26.0
792,Rajasthan,Phalodi,14.0,0.0,0.000000,14.0
795,Rajasthan,Salumbar,20.0,0.0,0.000000,20.0
804,Sikkim,Namchi,6.0,0.0,0.000000,6.0
918,Uttar Pradesh,Baghpat *,1.0,0.0,0.000000,1.0
934,Uttar Pradesh,Chandauli *,1.0,0.0,0.000000,1.0
1040,West Bengal,East Midnapur,2.0,0.0,0.000000,2.0
1086,West Bengal,nadia,1.0,0.0,0.000000,1.0


In [15]:
# Plot bio/demo ratio distribution
if 'bio_demo_ratio' in child_metrics.columns:
    fig = px.histogram(child_metrics.dropna(subset=['bio_demo_ratio']), x='bio_demo_ratio', nbins=50,
                       title='Distribution of Bio/Demo Ratio by District (Child 5-17)',
                       labels={'bio_demo_ratio': 'Biometric / Demographic Update Ratio'})
    fig.add_vline(x=1.0, line_dash='dash', line_color='red', annotation_text='Ideal (1.0)')
    fig.add_vline(x=0.5, line_dash='dash', line_color='orange', annotation_text='Bottleneck Threshold (0.5)')
    fig.show()

## 6. Data Issues Summary

In [16]:
# Compile data issues for logging
issues = []

# Issue 1: Date range mismatch
issues.append(f"Date range mismatch: Enrolment ({enrol_daily['date'].min().date()} to {enrol_daily['date'].max().date()}), "
              f"Biometric ({bio_daily['date'].min().date()} to {bio_daily['date'].max().date()})")

# Issue 2: District mismatches
issues.append(f"District name mismatches: {len(all_districts) - len(common_districts)} districts not in all 3 datasets")

# Issue 3: Age bucket alignment
issues.append("Age bucket alignment: Enrolment has 0-5 age group with no equivalent in update datasets")

# Issue 4: Bottleneck districts
if 'bio_demo_ratio' in child_metrics.columns:
    low_ratio_count = (child_metrics['bio_demo_ratio'] < 0.5).sum()
    issues.append(f"Potential bottleneck districts (bio/demo ratio < 0.5): {low_ratio_count}")

print("\n=== Data Issues Summary ===")
for i, issue in enumerate(issues, 1):
    print(f"{i}. {issue}")


=== Data Issues Summary ===
1. Date range mismatch: Enrolment (2025-03-02 to 2025-12-31), Biometric (2025-03-01 to 2025-12-29)
2. District name mismatches: 81 districts not in all 3 datasets
3. Age bucket alignment: Enrolment has 0-5 age group with no equivalent in update datasets
4. Potential bottleneck districts (bio/demo ratio < 0.5): 32


In [17]:
# Save issues to file
with open('../data/inventory/data_issues.md', 'w') as f:
    f.write('# Data Issues Log\n\n')
    f.write(f'Generated: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\n\n')
    for i, issue in enumerate(issues, 1):
        f.write(f'{i}. {issue}\n')
    
    # Add detailed district mismatches
    f.write('\n## District Mismatches (Samples)\n\n')
    f.write(f'Only in Enrolment: {list(only_in_enrol)[:10]}\n\n')
    f.write(f'Only in Demographic: {list(only_in_demo)[:10]}\n\n')
    f.write(f'Only in Biometric: {list(only_in_bio)[:10]}\n\n')

print("Issues saved to ../data/inventory/data_issues.md")

Issues saved to ../data/inventory/data_issues.md


In [18]:
# Create data inventory CSV
inventory = pd.DataFrame({
    'dataset': ['Enrolment', 'Demographic', 'Biometric'],
    'rows': [len(enrolment), len(demo_update), len(bio_update)],
    'columns': [len(enrolment.columns), len(demo_update.columns), len(bio_update.columns)],
    'date_min': [enrolment['date'].min(), demo_update['date'].min(), bio_update['date'].min()],
    'date_max': [enrolment['date'].max(), demo_update['date'].max(), bio_update['date'].max()],
    'states': [enrolment['state'].nunique(), demo_update['state'].nunique(), bio_update['state'].nunique()],
    'districts': [enrolment['district'].nunique(), demo_update['district'].nunique(), bio_update['district'].nunique()],
    'age_columns': [str(enrol_age_cols), str(demo_age_cols), str(bio_age_cols)]
})

inventory.to_csv('../data/inventory/data_inventory.csv', index=False)
print("Inventory saved to ../data/inventory/data_inventory.csv")
display(inventory)

Inventory saved to ../data/inventory/data_inventory.csv


,dataset,rows,columns,date_min,date_max,states,districts,age_columns
0,Enrolment,1006029,7,2025-03-02,2025-12-31,55,985,"['age_0_5', 'age_5_17', 'age_18_greater']"
1,Demographic,2071700,6,2025-03-01,2025-12-29,65,983,"['demo_age_5_17', 'demo_age_17_']"
2,Biometric,1861108,6,2025-03-01,2025-12-29,57,974,"['bio_age_5_17', 'bio_age_17_']"
